<a href="https://colab.research.google.com/github/financieras/math_for_ai/blob/main/estadistica/regresion_lineal_descenso_gradiente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Regresión Lineal con el algoritmo de descenso del gradiente**

---

## 1. Introducción: ¿Qué problema queremos resolver?

En el corazón del Machine Learning y la Ciencia de Datos se encuentra una tarea fundamental: la **predicción**. Queremos usar datos que ya tenemos para hacer estimaciones inteligentes sobre datos que aún no hemos visto.

Empecemos con un ejemplo clásico y sencillo: **predecir el precio de una vivienda basándonos en su tamaño.**

Imagina que tenemos un conjunto de datos de casas. Para cada casa, conocemos su tamaño en metros cuadrados (nuestra variable $x$) y el precio final por el que se vendió (nuestra variable $y$). Si visualizamos estos datos en un gráfico, probablemente veremos una "nube de puntos" que tiende a ir hacia arriba: a más metros cuadrados, mayor es el precio.



Nuestro objetivo es trazar **una línea recta** que represente de la mejor forma posible la tendencia de esos puntos. Esta línea será nuestro "modelo" de Regresión Lineal. ¿Por qué? Porque una vez que tengamos esa línea, si alguien nos da un nuevo tamaño ($x$) de una casa que no estaba en nuestros datos, podremos "consultar" la línea para estimar su precio ($y$).

### La Ecuación de Nuestro Modelo

Como recordarás de tus clases de matemáticas, la ecuación de una línea recta es $y = b + mx$. En Machine Learning, usamos una notación ligeramente diferente pero que significa exactamente lo mismo:

> $$\hat{y} = w_0 + w_1 x$$

Vamos a analizar estos términos, ya que los usaremos durante todo el artículo:

* **$x$**: Es nuestra variable de entrada (el *feature*), en este caso, el tamaño de la casa.
* **$\hat{y}$** (se pronuncia "y-sombrero" o "y-gorro"): Es la **predicción** de nuestro modelo (el precio estimado). La distinguimos de la $y$ real (el precio de venta verdadero).
* **$w_0$**: Es el **sesgo** (del inglés *bias*). Es la altura de la ordenada en el origen ($b$). Es el precio base que tendría nuestra predicción $\hat{y}$ si $x$ fuera 0.
* **$w_1$**: Es el **peso** (del inglés *weight*). Es el equivalente a la pendiente ($m$). Nos dice cuánto cambia $\hat{y}$ (precio) por cada unidad que aumenta $x$ (metro cuadrado).

**Objetivo:** Encontrar $ w_0 $ y $ w_1 $ que nos den la recta que mejor se ajuste a la nube de puntos.

La pregunta clave que da origen a todo lo que sigue es: De todas las rectas posibles, ¿cómo encontramos la que **"mejor se ajusta"** a los datos? ¿Qué significa "la mejor"?

Para responder a esto, necesitamos una forma de medir qué tan "equivocada" está nuestra línea. Necesitamos cuantificar el error. Y a esa medida la llamaremos nuestra **Función de Costes**.

---

## 2. 🎯 Midiendo el Error: La Función de Costes

En el apartado anterior, nos quedamos con una pregunta clave: ¿cómo definimos la "mejor" línea?

Intuitivamente, la mejor línea será aquella que esté **lo más cerca posible de todos los puntos de datos** al mismo tiempo. Necesitamos una forma de cuantificar esta "cercanía" total.

### El Residuo: El Error de un Solo Punto

Primero, veamos el error para un solo punto. Digamos que tenemos una casa (nuestro punto $i$-ésimo) que mide $x_i$ metros cuadrados y se vendió por un precio real $y_i$.

Si nuestra línea (definida por $w_0$ y $w_1$) predice un precio $\hat{y}_i$ para esa casa, el error para *ese punto* es simplemente la diferencia vertical entre el valor real y el valor predicho.

$$\text{Error}_i = e_i = \hat{y}_i - y_i$$

A esta diferencia la llamamos **"residuo"**.
* Si el punto real está por encima de la línea, el residuo es positivo.
* Si el punto real está por debajo de la línea, el residuo es negativo.

### Agregando el Error: El Error Cuadrático Medio (MSE)

Ahora, ¿cómo combinamos los residuos de *todos* nuestros puntos ($m$ puntos en total) en una sola métrica?

El primer impulso sería simplemente sumarlos. Pero esto es una mala idea: un residuo de +1000 y otro de -1000 se cancelarían mutuamente, haciendo parecer que nuestro modelo no tiene error, ¡cuando en realidad está fallando estrepitosamente en ambos puntos!

Para solucionar esto, hacemos dos cosas:

1.  **Elevamos cada residuo al cuadrado:** $e_i^2 = (\hat{y}_i - y_i)^2$.
    * Esto convierte todos los errores en números positivos (ej. $(-100)^2 = 10000$ y $(+100)^2 = 10000$). ¡Se acabaron las cancelaciones!
    * Además, **penaliza los errores grandes mucho más** que los pequeños. Un error de 10 se convierte en 100, pero un error de 2 solo se convierte en 4. Esto fuerza al modelo a evitar predicciones muy alejadas de la realidad.

2.  **Calculamos la media:** Sumamos todos estos errores al cuadrado y los dividimos por el número de puntos ($m$). Esto nos da el **Error Cuadrático Medio** (o *Mean Squared Error, MSE*).

Esta métrica es nuestra **Función de Costes**, que comúnmente se denota como $J$.

$$J(w_0, w_1) = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)^2$$

Si sustituimos $\hat{y}_i$ por la ecuación de nuestra línea, $(w_0 + w_1 x_i)$, obtenemos la fórmula completa:

$$J(w_0, w_1) = \frac{1}{m} \sum_{i=1}^{m} (w_0 + w_1 x_i - y_i)^2$$

> **Nota técnica:** En muchos libros verás esta fórmula con un $1/2m$ en lugar de $1/m$. (Ej. $J = \frac{1}{2m} \sum...$). Este $1/2$ se añade por pura conveniencia matemática. No cambia dónde está el mínimo, pero hace que la derivada (que usaremos en el siguiente paso) sea un poco más "limpia" al cancelar un 2 que aparece al derivar el término al cuadrado.

### Nuestro Nuevo Objetivo

¡Este es el punto clave! Fíjate en $J(w_0, w_1)$. Nuestros datos ($x$ e $y$) son fijos. Por lo tanto, el coste $J$ **no es una función de $x$**, sino una función de nuestros parámetros $w_0$ y $w_1$.

* Diferentes valores de $w_0$ y $w_1$ (diferentes líneas) nos darán un coste $J$ diferente.
* Una línea mala tendrá un coste $J$ muy alto.
* Una línea buena tendrá un coste $J$ muy bajo.

Si imaginamos todos los posibles valores de $w_0$ y $w_1$ y el coste $J$ que producen, obtendríamos una superficie en 3D con forma de "cuenco" o valle.



Nuestro problema de "encontrar la mejor línea" se ha transformado en un problema de optimización mucho más claro:

**Encontrar los valores de $w_0$ y $w_1$ que nos sitúen en el punto más bajo (el mínimo) de este cuenco.**

¿Y cómo encontramos ese punto mínimo? No lo haremos probando todas las combinaciones al azar. Usaremos un algoritmo inteligente llamado **Descenso del Gradiente**.

---

## 3. 📉 El Algoritmo: Descenso del Gradiente (Gradient Descent)

Ahora que sabemos que nuestro objetivo es minimizar la Función de Costes $J(w_0, w_1)$, necesitamos un método sistemático para alcanzar ese mínimo global. Aquí es donde entra en juego el **Descenso del Gradiente**.

El Descenso del Gradiente es un **algoritmo de optimización iterativo** que se utiliza para encontrar los valores de los parámetros $(w_0, w_1)$ que minimizan una función (nuestra función de costes).

### La Analogía de la Montaña ⛰️

La forma más intuitiva de entender el Descenso del Gradiente es a través de una analogía.

Imagina que estás en la cima de una montaña, con los ojos vendados, y tu objetivo es llegar al valle (el punto más bajo).

1.  **Tu Posición:** Tu posición actual en la montaña corresponde a los valores actuales de tus parámetros **$(w_0, w_1)$**.
2.  **El Objetivo:** El valle corresponde al **mínimo global** de la función de costes $J$.

Como estás vendado, no puedes ver el valle, pero puedes sentir el suelo bajo tus pies. ¿Cómo te mueves de manera eficiente?

* **Paso 1: Siente la Pendiente:** Tientas el suelo a tu alrededor para determinar la dirección en la que la pendiente es **más pronunciada hacia abajo**. Esta dirección de máximo descenso es el **gradiente**.
* **Paso 2: Da un Paso:** Una vez que conoces la dirección, das un paso. El tamaño de ese paso está determinado por la **tasa de aprendizaje**.
* **Paso 3: Repite:** Repites este proceso (sentir la pendiente y dar un paso) hasta que llegas a un punto donde ya no puedes bajar más.

El Descenso del Gradiente hace exactamente esto, pero en el mundo de las matemáticas:

### El Descenso del Gradiente en ML

El algoritmo comienza con unos valores **iniciales aleatorios** para nuestros parámetros $w_0$ y $w_1$ (estás en algún punto aleatorio de la montaña). Luego, repite un ciclo de actualización hasta la **convergencia**:

#### 1. Calcular el Gradiente (La Dirección)

El gradiente es una herramienta del cálculo (un vector de derivadas parciales) que nos dice exactamente cuál es la **pendiente** de la función de costes $J$ en nuestra posición actual $(w_0, w_1)$.

* Si la pendiente es positiva, significa que estamos a la izquierda del mínimo y debemos reducir el valor de $w$.
* Si la pendiente es negativa, estamos a la derecha y debemos aumentar el valor de $w$.

Matemáticamente, el gradiente apunta siempre hacia la **máxima subida**. Por lo tanto, si queremos *descender* (minimizar el coste), debemos movernos en la **dirección opuesta** al gradiente. Esto explica el signo negativo que introduciremos.

#### 2. La Actualización de los Parámetros (El Paso)

En cada iteración, actualizamos **simultáneamente** $w_0$ y $w_1$ usando la siguiente regla:

$$\text{Nuevo } w_j = \text{Antiguo } w_j - (\text{Tasa de Aprendizaje } \times \text{ Gradiente})$$

Donde $w_j$ representa cualquiera de nuestros parámetros ($w_0$ o $w_1$).

El signo de resta es lo que garantiza el "descenso": estamos moviéndonos en contra de la dirección de la pendiente.

Los detalles de cómo se calcula el gradiente y cómo se elige la tasa de aprendizaje son cruciales y se explican a continuación.

---

## 4. 🧩 Las Piezas Clave del Algoritmo

El Descenso del Gradiente es simple, pero su eficacia reside en la correcta aplicación de dos componentes esenciales: el **Gradiente** (la dirección de la pendiente) y la **Tasa de Aprendizaje** (el tamaño del paso).

### A. El Gradiente: La Dirección de Máximo Descenso

Como mencionamos, necesitamos calcular la pendiente de la función de costes $J(w_0, w_1)$ en nuestros valores de parámetros actuales. Esto se logra calculando las **derivadas parciales** de $J$ con respecto a cada parámetro ($w_0$ y $w_1$).

El resultado de estas derivadas nos dirá cuánto cambiaría el coste $J$ si modificáramos ligeramente un parámetro, manteniendo el otro fijo.

Recordemos la fórmula del coste MSE (usando la notación $1/m$):
$$J(w_0, w_1) = \frac{1}{m} \sum_{i=1}^{m} (w_0 + w_1 x_i - y_i)^2$$

#### Fórmulas de las Derivadas Parciales (El Gradiente)

Al aplicar las reglas de la derivación a la función de costes $J(w)$, obtenemos las siguientes expresiones para el gradiente:

1.  **Derivada con respecto a $w_0$ (el sesgo):**
    $$\frac{\partial J}{\partial w_0} = \frac{2}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)$$
    *Esta derivada es simplemente la media de todos los errores (la diferencia entre la predicción $\hat{y}$ y el valor real $y$).*

2.  **Derivada con respecto a $w_1$ (el peso/pendiente):**
    $$\frac{\partial J}{\partial w_1} = \frac{2}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i) \cdot x_i$$
    *Esta derivada es la media de los errores, multiplicada por la característica $x_i$. Esto significa que los puntos con valores de $x$ más grandes tendrán una mayor influencia en el ajuste de $w_1$.*

> **Simplificación:** Para la implementación, es común trabajar con la versión con $\frac{1}{2m}$ en la función de costes, lo que simplifica las fórmulas anteriores y elimina el factor 2:
> $$\frac{\partial J}{\partial w_0} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i)$$
> $$\frac{\partial J}{\partial w_1} = \frac{1}{m} \sum_{i=1}^{m} (\hat{y}_i - y_i) \cdot x_i$$

---

### B. La Tasa de Aprendizaje ($\alpha$): El Tamaño del Paso

La **tasa de aprendizaje** ($\alpha$) es un **hiperparámetro** fundamental que controla la magnitud de los pasos que damos en la dirección del gradiente. En el Machine Learning, los hiperparámetros son valores que debemos establecer *antes* de que el algoritmo comience a entrenar.

#### La Regla de Actualización

Con la dirección del gradiente y el tamaño del paso $\alpha$, definimos la regla de actualización. Esta se aplica repetidamente durante un número predefinido de iteraciones (llamadas **épocas**):

**Repetir (Iteraciones/Épocas) {**

$$\mathbf{w_0} := w_0 - \alpha \frac{\partial J}{\partial w_0}$$

$$\mathbf{w_1} := w_1 - \alpha \frac{\partial J}{\partial w_1}$$

**}**

#### El Impacto Crítico de $\alpha$

Elegir la $\alpha$ correcta es un acto de equilibrio:

| Si $\alpha$ es... | Consecuencia... | Gráfico conceptual... |
| :--- | :--- | :--- |
| **Demasiado Pequeña** | El algoritmo convergirá (llegará al mínimo), pero lo hará **extremadamente lento**. Se necesitarán miles de épocas, lo que consume mucho tiempo y recursos. |  |
| **Correcta** | El algoritmo converge eficientemente, acercándose al mínimo en un número razonable de pasos. |  |
| **Demasiado Grande** | El algoritmo puede **divergir** u **oscilar**. Al dar pasos gigantescos, el modelo salta de un lado a otro del valle, o incluso se aleja del mínimo, haciendo que el coste $J$ aumente con el tiempo. |  |

En la práctica, la $\alpha$ se ajusta mediante experimentación, siendo un valor típico inicial alrededor de $0.01$, $0.001$, o $0.0001$.

---

### C. Nota sobre el Cálculo Simultáneo

Es vital entender que, dentro de cada paso (cada época), las actualizaciones de $w_0$ y $w_1$ deben realizarse **simultáneamente**.

Esto significa que primero se deben calcular *ambas* derivadas (gradientes) utilizando los valores de $w_0$ y $w_1$ de la *iteración anterior*. Una vez que se tienen los dos gradientes, se actualizan ambos parámetros a sus nuevos valores. Si se actualizara $w_0$ y luego se usara el *nuevo* $w_0$ para calcular la derivada de $w_1$, se introduciría un sesgo en el algoritmo que podría llevar a resultados incorrectos.

## 5\. 👨‍💻 Manos a la Obra: Implementación "Manual" con Python y NumPy

Pasamos de la teoría a la práctica programando el algoritmo de **Regresión Lineal con Descenso de Gradiente (Gradient Descent)** desde cero. Usaremos las librerías NumPy para las operaciones matriciales eficientes y Matplotlib para la visualización.

-----

### 5.1. Importación y Preparación de Datos

Comenzamos importando las librerías necesarias y creando un conjunto de datos de ejemplo. Utilizaremos una relación lineal simple a la que añadiremos un poco de "ruido" aleatorio para simular datos reales.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Crear datos de ejemplo
np.random.seed(42) # Fijar semilla para reproducibilidad

# Variable independiente X (característica)
X = 2 * np.random.rand(100, 1) # 100 valores entre 0 y 2

# Variable dependiente y (objetivo)
# Relación real: y = 4 + 3 * X + ruido
y = 4 + 3 * X + np.random.randn(100, 1) * 1.5 # El ruido es esencial

# Visualizar
plt.figure(figsize=(8, 6))
plt.scatter(X, y, alpha=0.7, label='Datos reales')
plt.xlabel('Tamaño (normalizado)')
plt.ylabel('Precio')
plt.title('Datos de Entrenamiento')
plt.legend()
plt.grid(True)
plt.show()

-----

### 5.2. Escalamiento de Características (Estandarización)

**Importante:** Antes de iniciar el Descenso de Gradiente, es **crucial** realizar el **Escalado de Características** (Feature Scaling).

#### 💡 ¿Por qué es crucial el Escalado?

El Descenso de Gradiente funciona mejor cuando las características de entrada (las columnas de $X$) están en una escala similar. Si una característica (por ejemplo, el precio de una casa) tiene valores mucho mayores que otra (por ejemplo, el número de habitaciones), la función de costes ($J(w)$) será muy alargada y estrecha (una "elipse").

  * **Función de Costes Asimétrica:** Esto significa que las derivadas (el gradiente) serán muy grandes en la dirección de la característica con mayor escala.
  * **Convergencia Lenta o Divergencia:** El algoritmo de Descenso de Gradiente tendrá que dar "pasos" muy pequeños en la dirección de la característica con menor escala y "pasos" muy grandes y zigzagueantes en la dirección de la característica con mayor escala. Esto ralentiza drásticamente la convergencia o, en el peor de los casos, hace que el algoritmo no converja.

La **Estandarización** (o *Z-score normalization*) transforma los datos para que tengan una media ($\mu$) de 0 y una desviación estándar ($\sigma$) de 1.

$$X_{\text{estandarizado}} = \frac{X - \mu}{\sigma}$$

Aplicamos la estandarización solo a la característica $X$ (la segunda columna de `X_b`).

In [ ]:
# Calcular media y desviación estándar
mu = X.mean()
sigma = X.std()

print(f"Media (μ): {mu:.4f}")
print(f"Desviación estándar (σ): {sigma:.4f}")
print()

# Estandarizar: X' = (X - μ) / σ
X_scaled = (X - mu) / sigma

# Añadir columna de 1s para w0
# np.c_ concatena arrays por columnas
X_b = np.c_[np.ones((100, 1)), X_scaled]

X_b[:5] # Mostramos las 5 primeras filas de la matriz

Media (μ): 0.9404
Desviación estándar (σ): 0.5920



array([[ 1.        , -0.32311215],
       [ 1.        ,  1.62343393],
       [ 1.        ,  0.88450935],
       [ 1.        ,  0.43404902],
       [ 1.        , -1.06136481]])

-----

### 5.3. Implementación del Descenso de Gradiente

Ahora implementamos el núcleo del algoritmo, siguiendo la fórmula del Descenso de Gradiente.

**Fórmulas Clave:**

1.  **Predicción:** La predicción de los valores, denotada como $\hat{\mathbf{y}}$, se calcula matricialmente.  
    $$\hat{\mathbf{y}} = \mathbf{X} \mathbf{w}$$
2.  **Función de Coste (MSE):** La función de coste $J(\mathbf{w})$ se basa en el error cuadrático medio.  
    $$J(\mathbf{w}) = \frac{1}{2m} \sum_{i=1}^{m} (\hat{y}_i - y_i)^2$$
3.  **Regla de Actualización (Vectorial):** La actualización de los pesos se realiza en cada paso utilizando el gradiente.  
    $$\mathbf{w} := \mathbf{w} - \alpha \cdot \frac{1}{m} \cdot \mathbf{X}^{\text{T}} \cdot (\hat{\mathbf{y}} - \mathbf{y})$$

In [ ]:
# Hiperparámetros
learning_rate = 0.01    # alpha (tasa de aprendizaje)
epochs = 1000           # número de iteraciones
m = len(X)              # Número de ejemplos de entrenamiento

# Inicialización
w = np.zeros((2, 1))    # 'w' es el vector de pesos iniciales para [w_0, w_1]
cost_history = []       # (Opcional) Almacenar el historial de costes
w0_history = []
w1_history = []

# Bucle de entrenamiento (las epochs)
for epoch in range(epochs):
    # Predicciones (ŷ)
    # y_hat = X_scaled * w
    # Las predicciones son la regresión de regresión
    y_pred = X_b @ w        # dot product

    # Error
    errors = y_pred - y     # El error es (ŷ - y)

    # Gradiente: (1/m) * X_transpuesta * Errores
    gradients = (1/m) * X_b.T @ errors

    # Actualización: es w := w - alpha * gradiente
    w = w - learning_rate * gradients

    # Coste de cada paso (MSE/2)
    cost = (1/(2*m)) * np.sum(errors**2)
    cost_history.append(cost)

    # Guardar parámetros
    w0_history.append(w[0][0])
    w1_history.append(w[1][0])

# Pesos óptimos (en espacio estandarizado)
w_optimal = w
print(f"w0 (estandarizado): {w_optimal[0][0]:.6f}")
print(f"w1 (estandarizado): {w_optimal[1][0]:.6f}")

w0 (estandarizado): 6.819170
w1 (estandarizado): 1.571783


**Resultado de los parámetros óptimos:**

| Parámetro | Valor Óptimo (aproximado) |
| :---: | :---: |
| $w_0$ | $6.8192$ |
| $w_1$ | $1.5718$ |

-----

### 5.4. Visualización de Resultados

Para finalizar, ploteamos el historial de costes para confirmar que el algoritmo converge correctamente y visualizamos la línea de regresión final sobre los datos originales.

#### 5.4.1. Gráfico del Historial de Costes

El coste debe disminuir drásticamente al inicio y luego aplanarse, indicando la convergencia.

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(epochs), cost_history, 'b.')
plt.title('Historial de Costes')
plt.xlabel('Época (Epoch)')
plt.ylabel('Función de Coste J(w)')
plt.grid(True)
plt.show()

#### 5.4.2. Gráfico de la Regresión Lineal

Para plotear la línea de regresión, debemos usar los valores originales de $X$ (sin estandarizar) y aplicarles la misma estandarización antes de multiplicarlos por los $w$ óptimos.

In [ ]:
# Crear un rango de X para dibujar la línea
X_plot = np.array([[0], [2]])

# 1. Estandarizar X_plot de la misma forma que los datos de entrenamiento
X_plot_scaled = (X_plot - mu) / sigma

# 2. Añadir el sesgo (columna de unos)
X_plot_b = np.c_[np.ones((2, 1)), X_plot_scaled]

# 3. Calcular las predicciones con los pesos (w) óptimos
# Usamos w_optimal en lugar de theta_optimal
y_predict = X_plot_b.dot(w_optimal)

# Plotear los datos originales y la línea de regresión
plt.figure(figsize=(10, 6))
plt.plot(X, y, 'o', label='Datos originales')
plt.plot(X_plot, y_predict, 'r-', label='Línea de Regresión Final')
plt.title('Regresión Lineal con Descenso de Gradiente')
plt.xlabel('X (Característica Original)')
plt.ylabel('y (Objetivo)')
plt.legend()
plt.grid(True)
plt.show()

De esta forma, hemos implementado el Descenso de Gradiente de forma manual, demostrando cómo el algoritmo itera para encontrar los parámetros $w_0$ y $w_1$ que minimizan la función de costes y definen la línea de mejor ajuste.

## 6\. ⚡ La Vía Rápida: Implementación con Scikit-Learn

Tras haber programado la **Regresión Lineal con Descenso de Gradiente** manualmente, el objetivo de esta sección es mostrar cómo se realiza esta tarea en un **entorno profesional** utilizando la librería estándar de *machine learning* en Python: **Scikit-Learn (sklearn)**.

El modelo que utilizaremos es `sklearn.linear_model.SGDRegressor`.

-----

### 6.1. ¿Por qué `SGDRegressor`?

Mientras que en el apartado anterior implementamos el **Descenso de Gradiente por Lotes (Batch Gradient Descent)**, Scikit-Learn ofrece una variante mucho más común y eficiente para datos grandes: el **Descenso de Gradiente Estocástico (Stochastic Gradient Descent - SGD)**.

#### Concepto Clave: SGD

El SGD es una variante del Descenso de Gradiente donde, en lugar de calcular el gradiente usando **todos** los ejemplos de entrenamiento (*batch* completo) en cada paso, el algoritmo:

1.  Calcula el gradiente usando **un solo ejemplo** de entrenamiento seleccionado al azar (o un pequeño subconjunto llamado *mini-batch*).
2.  Actualiza los parámetros $w$ inmediatamente.

Esta aproximación hace que el proceso sea **mucho más rápido** en datasets con millones de datos, aunque el camino hacia el mínimo de la función de costes es más ruidoso y aleatorio. Para la regresión lineal, el `SGDRegressor` es la herramienta estándar cuando se desea aplicar el Descenso de Gradiente.

-----

### 6.2. Implementación con Scikit-Learn

A diferencia de la implementación manual, Scikit-Learn requiere que el escalado de características y el modelo se manejen como objetos separados.

#### 6.2.1. Preparación y Escalado de Datos

**Nota Importante:** Usaremos los datos originales $X$ e $y$ del apartado 5 para el proceso de escalado, asegurándonos de que $y$ tenga la forma correcta para Scikit-Learn.

In [ ]:
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np

# Datos originales (asumimos que X e y son los del apartado 5)
# X (100, 1), y (100, 1)

# 1. Ajustar la forma de 'y' si es necesario (Scikit-Learn prefiere (n_samples,))
y_flat = y.ravel()

# 2. Crear y ajustar el StandardScaler
# El escalador calculará la media y la desviación estándar de X
scaler = StandardScaler()
X_scaled_skl = scaler.fit_transform(X)
X_scaled_skl[:5]

array([[-0.32311215],
       [ 1.62343393],
       [ 0.88450935],
       [ 0.43404902],
       [-1.06136481]])

#### 6.2.2. Entrenamiento del Modelo `SGDRegressor`

Instanciamos y entrenamos el modelo. Es crucial especificar los hiperparámetros:

  * `loss='squared_error'`: Indica que queremos minimizar el Error Cuadrático Medio (MSE), adecuado para regresión lineal.
  * `eta0`: La tasa de aprendizaje inicial ($\alpha$).
  * `max_iter`: El número de épocas.

In [ ]:
# 3. Crear una instancia de SGDRegressor
sgd_reg = SGDRegressor(
    loss='squared_error',
    eta0=0.01,         # Learning rate (alpha)
    max_iter=1000,     # Número de épocas
    tol=1e-3,          # Criterio de parada
    random_state=42    # Para obtener resultados reproducibles
)

# 4. Entrenar el modelo con .fit(X_scaled, y)
sgd_reg.fit(X_scaled_skl, y_flat)

# 5. Mostrar los parámetros encontrados
# Renombramos las variables a w_0 y w_1 para mantener la consistencia
w_0_skl = sgd_reg.intercept_[0]
w_1_skl = sgd_reg.coef_[0]

print("\n--- Parámetros Óptimos encontrados por SGDRegressor ---")
# Usamos w_0 y w_1 en el texto impreso
print(f"w_0 (Intercepto): {w_0_skl:.4f}")
print(f"w_1 (Pendiente): {w_1_skl:.4f}")


--- Parámetros Óptimos encontrados por SGDRegressor ---
w_0 (Intercepto): 6.7827
w_1 (Pendiente): 1.5603


-----

### 6.3. Comparación de Resultados

Comparamos los resultados de la implementación manual (Descenso de Gradiente por Lotes) y la implementación profesional de Scikit-Learn (Descenso de Gradiente Estocástico).

| Parámetro | Manual (Batch GD) | Scikit-Learn (SGDRegressor) |
| :---: | :---: | :---: |
| $w_0$ (Intercepto) | $6.8192$ | **$6.7827$** |
| $w_1$ (Pendiente) | $1.5718$ | **$1.5603$** |


Los resultados son prácticamente **iguales**. Esto permite verificar dos puntos fundamentales:

1.  **Validación de la teoría:** Nuestra implementación manual funcionó correctamente.
2.  **Eficiencia de la herramienta:** En la práctica, el `SGDRegressor` nos permite obtener los mismos resultados óptimos con una fracción del código, beneficiándonos de la optimización y robustez de una librería profesional.

---

## 7. 🎯 Conclusión: ¿Qué Hemos Aprendido?

A lo largo de este artículo, hemos desglosado la **Regresión Lineal** desde sus cimientos matemáticos hasta su implementación práctica, comprendiendo que es mucho más que una simple línea de mejor ajuste. Los conceptos que hemos cubierto forman la base de la optimización en casi todo el campo del *Machine Learning*.

---

### Resumen de Puntos Clave

* **Objetivo de la Regresión Lineal:** La meta fundamental de la Regresión Lineal es encontrar los parámetros ($w_0$ y $w_1$) que definen la recta que mejor se ajusta a los datos.
* **La Función de Costes (MSE):** Para determinar qué tan "buena" es una recta, utilizamos una métrica de error, conocida comúnmente como el **Error Cuadrático Medio (MSE)** o $J(w)$. El verdadero objetivo del modelo es **minimizar** el valor de esta función.
* **El Algoritmo de Optimización: Descenso de Gradiente:** El **Descenso de Gradiente (*Gradient Descent*)** es el algoritmo que nos permite alcanzar ese mínimo. Podemos visualizarlo como un proceso iterativo en el que "caminamos" por la superficie de la función de costes.
    * **El Gradiente es la Brújula:** El gradiente (las derivadas parciales) indica la **dirección de máximo ascenso** en la función de costes. Puesto que queremos *minimizar* el coste, nuestro paso va en la dirección **opuesta** al gradiente.
    * **La Tasa de Aprendizaje ($\alpha$) es el Tamaño del Paso:** La **tasa de aprendizaje** determina la magnitud de cada paso. Si es muy grande, corremos el riesgo de "saltar" el mínimo; si es muy pequeña, la convergencia será extremadamente lenta.
* **Importancia del Escalado:** El **Escalado de Características** (como la Estandarización) es crucial para asegurar que la función de costes sea más simétrica, permitiendo que el Descenso de Gradiente converja de manera más **rápida y estable**.
* **Implementaciones (Manual vs. Scikit-Learn):** Hemos comprobado que, si bien es posible y educativo programar el algoritmo desde cero con NumPy (Descenso por Lotes), en un entorno profesional se utiliza `SGDRegressor` de Scikit-Learn (Descenso Estocástico), que ofrece la misma precisión con mayor eficiencia y rapidez en grandes volúmenes de datos.

---

### La Base de todo el *Machine Learning*

El concepto de **Descenso de Gradiente** no se limita a la Regresión Lineal. La idea de definir una función de costes, calcular su gradiente y ajustar parámetros de forma iterativa es el **motor de optimización** de la inmensa mayoría de los modelos de *Machine Learning* modernos, incluyendo:

* **Regresión Logística.**
* **Máquinas de Soporte Vectorial (SVM).**
* Y, de manera más notable, el **Entrenamiento de Redes Neuronales Profundas**, donde la técnica central es una aplicación sofisticada del Descenso de Gradiente llamada **Retropropagación (*Backpropagation*)**.

Entender el Descenso de Gradiente es, por lo tanto, entender **cómo aprende una máquina**.